In [ ]:
import logging
import random
from collections import Counter

import numpy as np
import pandas as pd
import spacy
from simpletransformers.classification import MultiLabelClassificationArgs
from simpletransformers.classification import MultiLabelClassificationModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sqlalchemy.orm import Session
from sqlalchemy.orm import joinedload
from tqdm.auto import tqdm

import src
import src.db.models.doccano as m
from src.coding.labels import LABELS
from src.db.connect import make_engine
from src.db.sample import Sample

In [ ]:
# setup
project = "PBert AnnoTask 5"

engine = make_engine("DOCCANO")

session = Session(engine)

pd.set_option("display.max_colwidth", 1024)
pd.set_option("display.max_rows", 512)

SEED = 109
random.seed(SEED)

logging.basicConfig(level=logging.ERROR)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

MODEL_PATH = src.PATH / "tmp/model_dir"
MODEL_PATH.mkdir(exist_ok=True, parents=True)

CACHE_DIR = src.PATH / "tmp/cache_dir"
CACHE_DIR.mkdir(exist_ok=True)

TENSORBOARD_DIR = src.PATH / "tmp/tensorboard"
TENSORBOARD_DIR.mkdir(exist_ok=True)

In [ ]:
# prepare sentence tokenizer

nlp = spacy.load("de_core_news_md")
nlp.disable_pipes(["tagger", "morphologizer", "lemmatizer", "attribute_ruler", "ner"])

['tagger', 'morphologizer', 'lemmatizer', 'attribute_ruler', 'ner']

In [ ]:
q = (
    session.query(m.ExamplesExample)
    .options(joinedload(m.ExamplesExample.labels), joinedload(m.ExamplesExample.state))
    .join(m.ExamplesExample.project)
    .filter(
        m.ProjectsProject.name == project,
        # uncomment to only collect samples confirmed by at least one person
        m.ExamplesExample.state.any(),
    )
)

label_dict = {label: LABELS[label]["lr"] for label in LABELS.keys()}
samples = []
for row in q:
    sample = Sample(row, nlp, label_dict)
    samples.append(sample)

In [ ]:
rows = []

for s in tqdm(samples):
    if "nicht zutr" in s.labels:
        continue

    # if "none" in s.labels and random.random() < 0.6:
    #    continue

    if s.labels == ["none"]:
        labels = []
    else:
        labels = s.labels

    rows.append((s.txt, labels))


df = pd.DataFrame(rows, columns=["text", "labels"])

mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(df["labels"])
num_labels = len(mlb.classes_)

df = pd.DataFrame(zip(df.text, labels), columns=["text", "labels"])

train, test = train_test_split(df, random_state=SEED, test_size=0.4, shuffle=True)

  0%|          | 0/1473 [00:00<?, ?it/s]

In [ ]:
train.shape, test.shape

((876, 2), (585, 2))

In [ ]:
# true values

print(f"TRAIN: {Counter(mlb.inverse_transform(np.array(train.labels.tolist())))}")
print(f"TEST: {Counter(mlb.inverse_transform(np.array(test.labels.tolist())))}")

TRAIN: Counter({(): 823, ('neutral',): 39, ('links',): 13, ('neutral', 'rechts'): 1})
TEST: Counter({(): 560, ('neutral',): 21, ('links',): 4})


In [ ]:
mlb.classes_

array(['links', 'neutral', 'rechts'], dtype=object)

In [ ]:
model_args = MultiLabelClassificationArgs(
    # do not change:
    output_dir=str(MODEL_PATH),
    cache_dir=str(CACHE_DIR),
    tensorboard_dir=str(TENSORBOARD_DIR),
    overwrite_output_dir=True,
    manual_seed=SEED,
    # hyperparameters to be optimized:
    num_train_epochs=7,
    train_batch_size=8,
    learning_rate=2e-5,
    # gradient_accumulation_steps=16,
)


weight_dict = {
    "none": 0.01,
    "neutral": 60,
    "links": 80,
    "rechts": 120,
}

pos_weights = [weight_dict.get(label, 1) for label in mlb.classes_]

model = MultiLabelClassificationModel(
    "bert",
    "bert-base-german-cased",
    num_labels=num_labels,
    pos_weight=pos_weights,
    args=model_args,
)

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForMultiLabelSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClassification were

In [ ]:
%%capture --no-display

model.train_model(train)

  0%|          | 0/876 [00:00<?, ?it/s]

Epoch:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 0 of 7:   0%|          | 0/110 [00:00<?, ?it/s]

Running Epoch 1 of 7:   0%|          | 0/110 [00:00<?, ?it/s]

Running Epoch 2 of 7:   0%|          | 0/110 [00:00<?, ?it/s]

Running Epoch 3 of 7:   0%|          | 0/110 [00:00<?, ?it/s]

Running Epoch 4 of 7:   0%|          | 0/110 [00:00<?, ?it/s]

Running Epoch 5 of 7:   0%|          | 0/110 [00:00<?, ?it/s]

Running Epoch 6 of 7:   0%|          | 0/110 [00:00<?, ?it/s]

(770, 1.6658595140467023)

In [ ]:
result, output, wrong_predictions = model.eval_model(test)

  0%|          | 0/585 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/74 [00:00<?, ?it/s]

In [ ]:
result

{'LRAP': 0.9931623931623932, 'eval_loss': 3.239476429801938}

In [ ]:
# predicted labels

pred_df = test.copy()

preds = model.predict(list(pred_df.text))
pred_labels = mlb.inverse_transform(np.array(preds[0]))

pred_df["y_pred"] = ["_".join(sorted(lab)) if lab else "none" for lab in pred_labels]
pred_df["y_true"] = [
    "_".join(sorted(lab)) if lab else "none"
    for lab in mlb.inverse_transform(np.array(pred_df.labels.tolist()))
]

  0%|          | 0/585 [00:00<?, ?it/s]

  0%|          | 0/74 [00:00<?, ?it/s]

In [ ]:
len(pred_df)

585

In [ ]:
# true values

Counter(mlb.inverse_transform(np.array(test.labels.tolist())))

Counter({(): 560, ('neutral',): 21, ('links',): 4})

In [ ]:
# predicted values
Counter(pred_labels)

Counter({(): 571, ('neutral',): 11, ('links',): 3})

## wrong predictions

In [ ]:
pred_df.loc[pred_df.y_pred != pred_df.y_true, ["y_pred", "y_true", "text"]].style.set_properties(
    subset="text", **{"text-align": "left"}
)

,y_pred,y_true,text
1297,neutral,none,"Es war die Bundeskanzlerin - und nicht die SPD-Fraktion -, die mit ihrer wohlkalkulierten Einlassung, dass dies allein eine Frage des Gewissens sei, die Türen für die heutige überstürzte Entscheidung sperrangelweit geöffnet hat und sich auch noch als quasi neue Fraktionsvorsitzende der Unionsfraktion dazu hat hinreißen lassen, generös die Abstimmung freizugeben, zudem auch noch entgegen dem eigenen, nach wie vor gültigen Grundsatzprogramm der CDU; denn darin steht zu lesen: Die Ehe ist unser Leitbild der Gemeinschaft von Mann und Frau. …Deshalb steht die Ehe unter dem besonderen Schutz unseres Grundgesetzes. Daran sehe ich: Beschlüsse der CDU sind offenkundig nicht das Papier wert, auf dem sie stehen."
1035,none,neutral,"Mit diesem Vorgehen wollen wir eine einheitliche Qualität im Berufsbild etablieren und die Angriffe der Altparteien obsolet machen, die auch beim MTA-Reform-Gesetz wieder sichtbar wurden. Denn diese wollen den ganzen Berufsstand abschaffen – sogar die Grünen, die dafür ihre eigene Stammwählerschaft vergraulen. Die AfD steht für eine freie Behandlungswahl und stellt sich mit diesem Antrag absolut gegen die Abschaffung des Heilpraktikers und für den Ausbau des Berufsbildes des Heilpraktikers."
890,none,neutral,"Denen ist längst klar: Diese SPD vertritt ihre Interessen nicht mehr. Die sagen sich zu Recht: Für die tut ihr alles, für uns tut ihr nichts. Das ist die SPD, die Heizöl und Benzin verteuern will, die einen Spurwechsel für abgelehnte Asylbewerber will, die für nur subsidiär Geschützte mehr Familiennachzug will und die die letztes Jahr nicht ausgeschöpfte Nachzugsquote dieses Jahr extra haben will."
287,none,neutral,"Ich glaube, Sie haben nicht die Mehrheit der Menschen in Deutschland hinter sich. Aber dieser Haushaltsentwurf ist nun wirklich einer, in dem die Bundesregierung ihre Maske hat fallen lassen. Er ist ein Haushalt des Übermaßes, und er ist ein Haushalt der Verantwortungslosigkeit unserem Land und der jungen Generation gegenüber."
885,none,neutral,"Eine Angabe im Pass ist … jedoch noch nicht möglich. Es kann … ein zusätzliches Dokument mit einem Hinweis, dass das Geschlecht als „X“ und somit als unspezifisch anzusehen ist, erteilt werden. Ich stelle fest: Die Bundesregierung hat keine Ahnung, wer die deutsche Grenze übertritt, sie kann aber in epischer Breite ausführen, wie es mit der Eintragung des dritten Geschlechtes auf Neufundland aussieht."
939,links,none,"Wir müssen unsere Bürger einfach fitter machen, damit sie mit ihren Qualifikationen den sich ändernden Bedingungen im globalen Wettbewerb besser entgegentreten können. Verantwortungsvolle Unternehmer machen das von ganz allein. Sie versuchen, ihre qualifizierten Mitarbeiter möglichst lange zu halten, und lagern nicht einfach alles in Billigjobs aus."
1356,neutral,none,Die Türkei redet hier von einer Selbstverteidigung. Man kann das aber mit guten Argumenten sicherlich auch als Angriff auf die territoriale Integrität Syriens begreifen. Bezeichnen Sie als Außenminister der neuen Bundesregierung ebenso wie die AfD-Fraktion den Einmarsch der Türkei in Syrien als völkerrechtswidrig?
733,none,neutral,"Was im Hause Andrea Nahles immer gern verschwiegen wird, ist ja, dass Sie auch Ausgaben für Gartenarbeiten als nicht regelsatzrelevant einstufen. Im Klartext heißt das: Erwerbslosen gestehen Sie nicht das Recht zu, in einem Nachbarschaftsgarten oder in einem Schrebergartenverein aktiv zu sein oder auf dem Balkon Tomaten zu züchten. Außerdem legen Sie fest: Ausgaben für Beherbergung sind nicht regelsatzrelevant."
325,none,neutral,"Das, was uns hier als Antrag der Linksfraktion zugemutet wird, ist aber von solch einer Qualität, die ich bisher wirklich kaum erlebt habe. Wenn man sich diesen Antrag anschaut, dann sieht man, dass er nur so von Unwahrheiten, Halbwahrheiten, illusorischen Forderungen und widersprüchlichen Dingen strotzt, und manchmal wird auch genau das gefordert, was wir bereits tun. Der Clou in di

## correct not-none predictions

In [ ]:
pred_df.loc[
    (pred_df.y_pred == pred_df.y_true) & (pred_df.y_true != "none"), ["y_pred", "y_true", "text"]
].style.set_properties(subset="text", **{"text-align": "left"})

,y_pred,y_true,text
1310,neutral,neutral,"Nur der unfassbaren Disziplin und Geduld der Deutschen ist es überhaupt zu verdanken, dass es noch keine flächendeckenden Massenproteste gibt wie in Frankreich, wo es den sogenannten Aufstand der gelben Westen gegen die neueste Spritpreiserhöhung gibt. Noch nie in der Geschichte der Bundesrepublik Deutschland hatte die öffentliche Hand so viel vom Bürger zur Verfügung, und noch nie wurde so viel Geld so schlecht ausgegeben. Wir haben eine vernachlässigte Armee, die ihren Auftrag der Landesverteidigung nicht erfüllen kann, deren Panzer nicht fahren, deren Schiffe nicht in See stechen, deren Flugzeuge und Hubschrauber nicht fliegen und die trotzdem in immer neue Einsätze in aller Welt geschickt wird."
1224,neutral,neutral,"Jetzt wird ihnen seitens ihrer ehemaligen Partei alles Mögliche unterstellt und geradezu eine demokratische Gesinnung abgesprochen. Wenn seitens der CDU so dramatische Demokratiedefizite bei diesen Männern erkannt werden, dass sie hier nicht einmal mehr als Alterspräsidenten fungieren können: Warum wurde dann nicht reagiert, als sie noch Mitglieder der CDU gewesen sind? Das wäre doch das Normalste von der Welt."
140,neutral,neutral,"Gerade als Naturwissenschaftlerin hätten Sie die Debatte über den Diesel auf die Sachebene holen müssen. Stattdessen haben Sie den grünen Autofeinden freien Lauf gelassen. Ihre Politik ist eine, in der Gesinnung statt Verantwortung, Gefühl statt Wissenschaft, Glaube statt Wissen dominiert haben."
1048,neutral,neutral,"Sie von den Grünen wollen immer nur den Sozialleistungsbeziehern die Rechte stärken und die Pflichten mindern, den Steuerzahlern hingegen bürden Sie damit immer mehr Pflichten und Belastungen auf. Sie sorgen dafür, dass es attraktiver wird, sich im Sozialstaat einzurichten und auf Kosten der Allgemeinheit zu leben. Damit zerstören Sie die Leistungsbasis der Gesellschaft und die Zukunftsfähigkeit des Sozialstaates."
521,neutral,neutral,"Offensichtlich hat die Regierung es einfach verschlafen. Ein leistungsfähiger Internetzugang gehört heute zur Grundversorgung. Meine Damen und Herren, werfen Sie also den ideologischen Ballast der sogenannten Verkehrswende ab, und kümmern Sie sich um die Dinge, die für die Bürger wirklich gut und wichtig sind."
542,neutral,neutral,"Die FDP hat dazu beigetragen, eine Stimmung zu schaffen, die öffentliche Behörden und Einrichtungen als Ursache allen Übels diffamiert. Durch die Verkleinerung, Zerlegung und Privatisierung wichtiger Bereiche der öffentlichen Daseinsvorsorge wurde die Fähigkeit des Staates geschwächt, soziale Prozesse demokratisch zu lenken. Damit haben Sie auch die Fähigkeit des Gesundheitssystems geschwächt, auf Krisen wie eine Pandemie zu reagieren."
863,neutral,neutral,"Deshalb steht die Ehe unter dem besonderen Schutz unseres Grundgesetzes. Daran sehe ich: Beschlüsse der CDU sind offenkundig nicht das Papier wert, auf dem sie stehen. Insgesamt ist der heutige Vorgang an Peinlichkeit kaum zu überbieten."
